<a href="https://www.kaggle.com/code/venkatasubramani/data-analysis-of-my-chess-com-games?scriptVersionId=134056976" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session8

In [2]:
from urllib.request import urlopen
import re
import json
import pandas as pd
import numpy as np
import warnings
import datetime
from datetime import datetime
warnings.filterwarnings("ignore")
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import time
user = 'venkykasprov'
url = "https://api.chess.com/pub/player/"+user+"/games/archives"
response = urlopen(url)
archives_json = json.loads(response.read())
print(archives_json)

# df = pd.DataFrame(data_json.items())

{'archives': ['https://api.chess.com/pub/player/venkykasprov/games/2015/02', 'https://api.chess.com/pub/player/venkykasprov/games/2015/03', 'https://api.chess.com/pub/player/venkykasprov/games/2015/04', 'https://api.chess.com/pub/player/venkykasprov/games/2015/07', 'https://api.chess.com/pub/player/venkykasprov/games/2015/10', 'https://api.chess.com/pub/player/venkykasprov/games/2015/11', 'https://api.chess.com/pub/player/venkykasprov/games/2015/12', 'https://api.chess.com/pub/player/venkykasprov/games/2016/12', 'https://api.chess.com/pub/player/venkykasprov/games/2019/01', 'https://api.chess.com/pub/player/venkykasprov/games/2019/02', 'https://api.chess.com/pub/player/venkykasprov/games/2019/03', 'https://api.chess.com/pub/player/venkykasprov/games/2019/04', 'https://api.chess.com/pub/player/venkykasprov/games/2019/06', 'https://api.chess.com/pub/player/venkykasprov/games/2019/07', 'https://api.chess.com/pub/player/venkykasprov/games/2019/08', 'https://api.chess.com/pub/player/venkyka

In [3]:
# json.loads(urlopen("https://api.chess.com/pub/player/venkykasprov/games/2020/08").read())

In [4]:
mygames_df = pd.DataFrame([])
for i in archives_json['archives']:
    print(i)
    month_json = json.loads(urlopen(i).read())
    month_df = pd.json_normalize(month_json['games'])
    mygames_df = pd.concat([mygames_df,month_df])
mygames_df = mygames_df.reset_index(drop=True)    
mygames_df.head()
mygames_df.to_csv('venkykasprov_games.csv')

https://api.chess.com/pub/player/venkykasprov/games/2015/02
https://api.chess.com/pub/player/venkykasprov/games/2015/03
https://api.chess.com/pub/player/venkykasprov/games/2015/04
https://api.chess.com/pub/player/venkykasprov/games/2015/07
https://api.chess.com/pub/player/venkykasprov/games/2015/10
https://api.chess.com/pub/player/venkykasprov/games/2015/11
https://api.chess.com/pub/player/venkykasprov/games/2015/12
https://api.chess.com/pub/player/venkykasprov/games/2016/12
https://api.chess.com/pub/player/venkykasprov/games/2019/01
https://api.chess.com/pub/player/venkykasprov/games/2019/02
https://api.chess.com/pub/player/venkykasprov/games/2019/03
https://api.chess.com/pub/player/venkykasprov/games/2019/04
https://api.chess.com/pub/player/venkykasprov/games/2019/06
https://api.chess.com/pub/player/venkykasprov/games/2019/07
https://api.chess.com/pub/player/venkykasprov/games/2019/08
https://api.chess.com/pub/player/venkykasprov/games/2019/09
https://api.chess.com/pub/player/venkyka

In [5]:
mygames_df['my_colour']=np.where(mygames_df['white.username']==user,'white','black')
mygames_df['my_result']=np.where(mygames_df['white.username']==user,mygames_df['white.result'],mygames_df['black.result'])
mygames_df['my_rating']=np.where(mygames_df['white.username']==user,mygames_df['white.rating'],mygames_df['black.rating'])
mygames_df['opponent_rating']=np.where(mygames_df['white.username']!=user,mygames_df['white.rating'],mygames_df['black.rating'])
mygames_df['opponent_username']=np.where(mygames_df['white.username']!=user,mygames_df['white.username'],mygames_df['black.username'])
mygames_df['end_time'] = mygames_df['end_time'].astype('int').astype("datetime64[s]")
mygames_df['year'] = pd.to_datetime(mygames_df['end_time']).dt.year
mygames_df['result']='lose'
mygames_df['result'] = np.where(mygames_df['my_result']=='win','win',mygames_df['result'])
mygames_df['result'] = np.where((mygames_df['my_result']=='repetition')|(mygames_df['my_result']=='insufficient')|(mygames_df['my_result']=='timevsinsufficient')|(mygames_df['my_result']=='agreed')|(mygames_df['my_result']=='stalemate'),'draw',mygames_df['result'])
# mygames_df['my_colour']=np.where(mygames_df['white.username']==user)
mygames_df.head()

,url,pgn,time_control,end_time,rated,tcn,uuid,initial_setup,fen,time_class,...,start_time,accuracies.white,accuracies.black,my_colour,my_result,my_rating,opponent_rating,opponent_username,year,result
0,https://www.chess.com/game/live/1051463451,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",300,2015-02-06 11:17:31,True,mC0KgvZRfA3VbsWOlBKBdB!Teg5QBdQGArGrkr6ZsJTJCJ...,2506eeee-5dd6-11df-8000-000000010001,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,8/P5pk/7p/1r6/1P2R3/6P1/1P3P1P/6K1 b - -,blitz,...,NaN,NaN,NaN,white,win,1367,1208,rnsrayudu,2015,win
1,https://www.chess.com/game/live/1051466465,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",300,2015-02-06 11:22:16,True,mCYInD0KgvKDfAZReg2Mlt1TvMTMdN8ZN1701N3Vbs!TNx...,95ae0b0a-5dd6-11df-8000-000000010001,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,4r3/1pkq4/p2p3p/2pP2p1/2P1np2/4bR1P/PPP3P1/2Q3...,blitz,...,NaN,NaN,NaN,black,win,1497,1362,jerry_calvz,2015,win
2,https://www.chess.com/game/live/1051469304,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",300,2015-02-06 11:29:20,True,kA0SlB!TbsZJgv5ZmuYIiq90BIZIjzIZcj8!fmJAmAXPeg...,ff2e19d0-5dd6-11df-8000-000000010001,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,6k1/5p1p/pp2p1p1/8/1P1Nn3/r3PP1P/6P1/5RK1 b - -,blitz,...,NaN,NaN,NaN,black,win,1614,1537,MCSKDC,2015,win
3,https://www.chess.com/game/live/1051505078,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",300,2015-02-06 12:23:13,True,mCZJCJ7JbsJKdm5QgvKSmS6SfH!THQXQeg3Npx86ltTJsJ...,33f09fbc-5ddc-11df-8000-000000010001,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,2kr3r/B1p5/3bRp2/7p/6pP/6P1/bP3P2/3R2K1 b - -,blitz,...,NaN,NaN,NaN,white,resigned,1524,1612,albipapatr,2015,lose
4,https://www.chess.com/game/live/1051507389,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",300,2015-02-06 12:30:58,True,jr0Scj!Tbs90mC8!CKTJsJSJpFZRFNRKjK3Vdv98ec0TKs...,89c54802-5ddc-11df-8000-000000010001,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,r5k1/2p2pp1/7p/7P/7R/1P1P1P2/PK3q2/3R2N1 w - -,blitz,...,NaN,NaN,NaN,black,win,1615,1602,albipapatr,2015,win


In [6]:
mygames_df.groupby(['time_class','year'])['time_class'].count()

time_class  year
blitz       2015      53
            2016      22
            2019     288
            2020     869
            2021     386
            2022    1240
            2023     618
bullet      2015      18
            2016       2
            2019      76
            2020     352
            2021    1362
            2022     576
            2023     215
daily       2016       1
            2019       4
            2020       2
            2022       6
            2023       3
rapid       2020      48
            2021     298
            2022     168
            2023      29
Name: time_class, dtype: int64

In [7]:
mygames_df.groupby('rules')['rules'].count()

rules
bughouse          207
chess            6426
chess960            1
crazyhouse          1
kingofthehill       1
Name: rules, dtype: int64

In [8]:
mybughuse_games_df = mygames_df[mygames_df['rules']=='bughouse']
mybughuse_games_df

,url,pgn,time_control,end_time,rated,tcn,uuid,initial_setup,fen,time_class,...,start_time,accuracies.white,accuracies.black,my_colour,my_result,my_rating,opponent_rating,opponent_username,year,result
71,https://www.chess.com/game/live/1853537862,NaN,180,2016-12-14 17:20:07,True,mCYIfA0Sgv5Q=JSJAJ!Tbs+SsH=Y=A=OHsSJCJQBvBIBsm...,556eca3c-25c3-11e1-8000-000000010001,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,r1bqkb1r/pppp1ppp/p4nr1/3Pp3/2Pp3n/5PBP/PPPPNP...,blitz,...,NaN,NaN,NaN,black,bughousepartnerlose,1405,1745,denque,2016,lose
72,https://www.chess.com/game/live/1853543204,NaN,180,2016-12-14 17:20:22,True,mC,1c832528-25c4-11e1-8000-000000010001,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...,blitz,...,NaN,NaN,NaN,white,bughousepartnerlose,1405,1283,mukundpas,2016,lose
73,https://www.chess.com/game/live/1853544379,NaN,180,2016-12-14 17:21:13,True,mCYI,48060fee-25c4-11e1-8000-000000010001,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,rnbqkbnr/pp1ppppp/8/2p5/4P3/8/PPPP1PPP/RNBQKBN...,blitz,...,NaN,NaN,NaN,black,win,1405,1096,rohu333,2016,win
74,https://www.chess.com/game/live/1853544761,NaN,180,2016-12-14 17:26:17,True,mC!TbsZJCJTJsJ7JkAJC-u=t=vCKft5QgmQz+kzktk=xox...,56542bda-25c4-11e1-8000-000000010001,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,r3kb1r/ppp1pppp/8/4q3/2P5/5P2/PPBPNPpP/R1BQK1q...,blitz,...,NaN,NaN,NaN,white,timeout,1289,1622,black-fyra,2016,lose
75,https://www.chess.com/game/live/1853551903,NaN,180,2016-12-14 17:26:47,True,,602a5f16-25c5-11e1-8000-000000010001,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,blitz,...,NaN,NaN,NaN,black,win,1289,1860,DominikValledor,2016,win
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6577,https://www.chess.com/game/live/77640120485,NaN,180,2023-05-12 14:18:37,True,mC0SnD!0DLSLCL0LftLRdm=S-J=QJD90mN=UNx*Fxv+Evn&d,9fffa103-f0cf-11ed-99cf-6cfe544c0428,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,rnbqk2r/ppppbppp/2pnp1p1/8/5Nbr/3B4/PPPP1QPP/R...,blitz,...,NaN,NaN,NaN,black,win,1377,1237,SkolRook,2023,win
6578,https://www.chess.com/game/live/77640191379,NaN,180,2023-05-12 14:19:22,True,lt!TkA2U,ec113335-f0cf-11ed-99cf-6cfe544c0428,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,rnbqkb1r/pppppp1p/5np1/8/2P5/3P4/PP2PPPP/RNBQK...,blitz,...,NaN,NaN,NaN,white,win,1378,739,austintsai85,2023,win
6579,https://www.chess.com/game/live/77640216329,NaN,180,2023-05-12 14:24:25,True,lt0KkA9ImuZR-C6LCIRI=CLS=s5Q+JSJAJQ0gv1TfmYQeg...,06cd12ee-f0d0-11ed-99cf-6cfe544c0428,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,r2q1r1k/p1n4p/1p3p1Q/2p1pP1p/2N4p/2PP1N1P/PP2B...,blitz,...,NaN,NaN,NaN,white,bughousepartnerlose,1356,972,karthik_chessmaster,2023,lose
6580,https://www.chess.com/game/live/77640652897,NaN,180,2023-05-12 14:27:33,True,mC0Kgv5Qbs!TfA9zegzsls+N=m*yAtyCtCTC+J*HJC,bd467b71-f0d0-11ed-99cf-6cfe544c0428,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,r1bqk2r/pppp1ppp/2n5/1r2p2b/4B3/2P2N2/PPP1PPPP...,blitz,...,NaN,NaN,NaN,white,win,1356,451,BIGBUCKSLAYER2009,2023,win


In [9]:
mychess_games_df = mygames_df[mygames_df['rules']=='chess']
print(mychess_games_df.columns)
mychess_games_df.head(2)

Index(['url', 'pgn', 'time_control', 'end_time', 'rated', 'tcn', 'uuid',
       'initial_setup', 'fen', 'time_class', 'rules', 'white.rating',
       'white.result', 'white.@id', 'white.username', 'white.uuid',
       'black.rating', 'black.result', 'black.@id', 'black.username',
       'black.uuid', 'tournament', 'start_time', 'accuracies.white',
       'accuracies.black', 'my_colour', 'my_result', 'my_rating',
       'opponent_rating', 'opponent_username', 'year', 'result'],
      dtype='object')


,url,pgn,time_control,end_time,rated,tcn,uuid,initial_setup,fen,time_class,...,start_time,accuracies.white,accuracies.black,my_colour,my_result,my_rating,opponent_rating,opponent_username,year,result
0,https://www.chess.com/game/live/1051463451,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",300,2015-02-06 11:17:31,True,mC0KgvZRfA3VbsWOlBKBdB!Teg5QBdQGArGrkr6ZsJTJCJ...,2506eeee-5dd6-11df-8000-000000010001,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,8/P5pk/7p/1r6/1P2R3/6P1/1P3P1P/6K1 b - -,blitz,...,NaN,NaN,NaN,white,win,1367,1208,rnsrayudu,2015,win
1,https://www.chess.com/game/live/1051466465,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",300,2015-02-06 11:22:16,True,mCYInD0KgvKDfAZReg2Mlt1TvMTMdN8ZN1701N3Vbs!TNx...,95ae0b0a-5dd6-11df-8000-000000010001,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,4r3/1pkq4/p2p3p/2pP2p1/2P1np2/4bR1P/PPP3P1/2Q3...,blitz,...,NaN,NaN,NaN,black,win,1497,1362,jerry_calvz,2015,win


In [10]:
!pip install urlextract

In [11]:
def find_insights(mychess_games_df):
#     print(mychess_games_df['pgn'].head(1).values)
    string = mychess_games_df['pgn'].head(1).values[0].split('\n')[-2]
    time_pattern = re.compile(r'\d+:\d+:\d+(\.\d+)?')
    time_moves = []
    for match in re.finditer(time_pattern,string ):
        time = match.group()
        time_moves.append(time)
    if time_moves != []:
        white_pattern = r'\s*(\d{1,3})\.\s\s?\s*((?:(?:O-O(?:-O)?)|(?:[KQNBR][1-8a-h]?x?[a-h]x?[1-8])|(?:[a-h]x?[a-h]?[1-8]\=?[QRNB]?))\+?)'
        white_moves = []
        for match in re.finditer(white_pattern,string ):
        #     print(match)
        #     move_num = match.group(1)
            white_move = match.group(2)
            white_moves.append(white_move)
        pattern = r'\s*(\d{1,3})\.{3}\s?\s*((?:(?:O-O(?:-O)?)|(?:[KQNBR][1-8a-h]?x?[a-h]x?[1-8])|(?:[a-h]x?[a-h]?[1-8]\=?[QRNB]?))\+?)'
        black_moves = []
        for match in re.finditer(pattern,string ):
        #     print(match)
        #     move_num = match.group(1)
            black_move = match.group(2)
            black_moves.append(black_move)
#         print(black_moves,white_moves)
#         print(time_moves)
        if len(white_moves)!=len(black_moves):
            black_moves.append(' ')
            if len(white_moves)>1:
                time_moves.append(time_moves[-2])
        white_time =  pd.to_datetime(time_moves[::2])
        black_time =  pd.to_datetime(time_moves[1::2])
        mychess_games_df['white_avg_time_per_move'] =  ((white_time[0]-white_time[-1])/len(white_time)).total_seconds()
        if len(white_moves)==1:
            mychess_games_df['black_avg_time_per_move']= -1
        elif len(white_moves)!=len(black_moves):
            mychess_games_df['black_avg_time_per_move'] = ((black_time[0]-black_time[-1])/(len(black_time)-1)).total_seconds()
        else:
            mychess_games_df['black_avg_time_per_move'] = ((black_time[0]-black_time[-1])/len(black_time)).total_seconds()

        mychess_games_df['captured_black_pieces_count'] = sum([a.count('x') for a in white_moves])
        mychess_games_df['captured_white_pieces_count'] = sum([a.count('x') for a in black_moves])

        mychess_games_df['black_king_checks'] = sum([a.count('+') for a in white_moves])
        mychess_games_df['white_king_checks'] = sum([a.count('+') for a in black_moves])
    else:
        mychess_games_df['white_avg_time_per_move'] = -2
        mychess_games_df['black_avg_time_per_move'] =-2
        mychess_games_df['captured_black_pieces_count'] = 0
        mychess_games_df['captured_white_pieces_count'] = 0

        mychess_games_df['black_king_checks'] = 0
        mychess_games_df['white_king_checks'] = 0
        
    # moves_df = pd.DataFrame(columns=['white_moves','black_moves','white_time','black_time'])
    # moves_df['white_moves'] = np.array(white_moves)
    # moves_df['black_moves'] = np.array(black_moves)
    # moves_df['white_time'] = np.array(white_time)
    # moves_df['black_time'] = np.array(black_time)
    # moves_df
    return mychess_games_df

In [12]:
mychess_games_df['rating_class']='high'
mychess_games_df['rating_class'] = np.where((mychess_games_df['opponent_rating']>1200)&(mychess_games_df['opponent_rating']<=1800),'medium',mychess_games_df['rating_class'])
mychess_games_df['rating_class'] = np.where(mychess_games_df['opponent_rating']<=1200,'low',mychess_games_df['rating_class'])

In [13]:
rapid_time_series = mychess_games_df[mychess_games_df['time_class']=='rapid'][['end_time','my_rating']]
rapid_time_series = rapid_time_series.set_index('end_time')
fig = px.line(x=rapid_time_series.groupby("end_time")["my_rating"].mean().keys(), y=rapid_time_series.groupby("end_time")["my_rating"].mean().values,line_shape='hv', markers=True)
fig.update_layout(height=400,width=400,title_text="Rapid rating timeseries", xaxis={'title':'date and time'}, yaxis={'title':'rapid rating'})
fig.show()

In [14]:
blitz = mychess_games_df[mychess_games_df['time_class']=='rapid'][['end_time','my_rating','result']]
fig = px.line(x=blitz.groupby("end_time")["my_rating"].mean().keys(), y=blitz.groupby("end_time")["my_rating"].mean().values, markers=True,line_shape='hv',color=blitz['result'])
fig.update_layout(height=400,width=400,title_text="Rapid rating timeseries based on result", xaxis={'title':'date and time'}, yaxis={'title':'rapid rating'})
fig.show()

In [15]:
blitz_time_series = mychess_games_df[mychess_games_df['time_class']=='blitz'][['end_time','my_rating']]
blitz_time_series = blitz_time_series.set_index('end_time')
fig = px.line(x=blitz_time_series.groupby("end_time")["my_rating"].mean().keys(), y=blitz_time_series.groupby("end_time")["my_rating"].mean().values, markers=True,line_shape='hv')
fig.update_layout(height=400,width=400,title_text="Blitz rating timeseries", xaxis={'title':'date and time'}, yaxis={'title':'blitz rating'})
fig.show()

In [16]:
blitz = mychess_games_df[mychess_games_df['time_class']=='blitz'][['end_time','my_rating','result']]
fig = px.line(x=blitz.groupby("end_time")["my_rating"].mean().keys(), y=blitz.groupby("end_time")["my_rating"].mean().values, markers=True,line_shape='hv',color=blitz['result'])
fig.update_layout(height=400,title_text="Blitz rating timeseries based on result", xaxis={'title':'date and time'}, yaxis={'title':'blitz rating'})
fig.show()

In [17]:
bullet_time_series = mychess_games_df[mychess_games_df['time_class']=='bullet'][['end_time','my_rating']]
bullet_time_series = bullet_time_series.set_index('end_time')
fig = px.line(x=bullet_time_series.groupby("end_time")["my_rating"].mean().keys(), y=bullet_time_series.groupby("end_time")["my_rating"].mean().values, markers=True,line_shape='hv')
fig.update_layout(height=400,width=400,title_text="Bullet rating timeseries", xaxis={'title':'date and time'}, yaxis={'title':'bullet rating'})
fig.show()

In [18]:
bullet_time_series = mychess_games_df[mychess_games_df['time_class']=='bullet'][['end_time','my_rating']]
# bullet_time_series = bullet_time_series.set_index('end_time')
bullet_time_series['differ'] = bullet_time_series['my_rating'].diff()
colors = ['red' if x < 0 else 'green' for x in bullet_time_series['differ']]

# create a line plot with conditional coloring
fig = go.Figure()
fig.add_trace(go.Scatter(x=bullet_time_series['end_time'], y=bullet_time_series['my_rating'], mode='lines', line=dict(color='green')))
fig.add_trace(go.Scatter(x=bullet_time_series['end_time'], y=bullet_time_series['my_rating'].where(bullet_time_series.differ <0), mode='lines', line=dict(color='red')))
fig.update_layout(title='Line Plot with Conditional Coloring', xaxis_title='end_time', yaxis_title='my_rating')
fig.show()

bullet_time_series

,end_time,my_rating,differ
22,2015-10-16 07:12:36,1062,NaN
23,2015-10-16 07:15:34,896,-166.0
24,2015-10-16 07:18:07,982,86.0
25,2015-10-16 07:20:30,1037,55.0
26,2015-10-16 07:22:47,1122,85.0
...,...,...,...
6609,2023-05-26 14:08:21,1438,-8.0
6610,2023-05-26 14:10:45,1430,-8.0
6611,2023-05-26 14:13:02,1437,7.0
6612,2023-05-29 10:08:36,1430,-7.0


In [19]:
bullet = mychess_games_df[mychess_games_df['time_class']=='bullet'][['end_time','my_rating','result']].reset_index(drop=True)
bullet['end_time']=pd.to_datetime(bullet['end_time']).dt.date
bullet['end_time'] = pd.to_datetime(bullet['end_time'])
bullet_t = bullet.groupby('end_time')['my_rating'].agg(['min', 'max', 'first', 'last']).reset_index()
# bullet_t = bullet[['end_time']].merge(bullet_t, on='end_time')
bullet_t.set_index('end_time', inplace=True)
bullet_t = bullet_t.resample('D').ffill().reset_index()

# bullet_t['min']= bullet_t['min'].ffill()
# bullet_t['max']= bullet_t['max'].ffill()
# bullet_t['first']= bullet_t['first'].ffill()
# bullet_t['last']= bullet_t['last'].ffill()
print(bullet_t)

fig = go.Figure(data=[go.Candlestick(x=bullet_t['end_time'], open=bullet_t['first'], high=bullet_t['max'], low=bullet_t['min'], close=bullet_t['last'])])
fig.show()


       end_time   min   max  first  last
0    2015-10-16   896  1172   1062  1172
1    2015-10-17   896  1172   1062  1172
2    2015-10-18   896  1172   1062  1172
3    2015-10-19   896  1172   1062  1172
4    2015-10-20  1131  1181   1131  1181
...         ...   ...   ...    ...   ...
2779 2023-05-26  1430  1446   1446  1437
2780 2023-05-27  1430  1446   1446  1437
2781 2023-05-28  1430  1446   1446  1437
2782 2023-05-29  1430  1430   1430  1430
2783 2023-05-30  1422  1422   1422  1422

[2784 rows x 5 columns]


In [20]:
bullet = mychess_games_df[mychess_games_df['time_class']=='bullet'][['end_time','my_rating','result']]

fig = px.line(x=bullet.groupby("end_time")["my_rating"].mean().keys(), y=bullet.groupby("end_time")["my_rating"].mean().values, markers=True,line_shape='hv',color=bullet['result'])
fig.update_layout(height=400,width=400,title_text="Bullet rating timeseries based on result", xaxis={'title':'date and time'}, yaxis={'title':'blitz rating'})
fig.show()

In [21]:
mychess_games_df['time_class'].unique()

array(['blitz', 'bullet', 'daily', 'rapid'], dtype=object)

In [22]:
mychess_games_df = mychess_games_df[mychess_games_df['time_class'].isin(['blitz', 'bullet', 'rapid'])].reset_index(drop=True)
mychess_games_df = mychess_games_df.groupby(['pgn']).apply(find_insights).reset_index()
# pip install chess

In [23]:
from urlextract import URLExtract

extractor = URLExtract()
urls = extractor.find_urls(mychess_games_df['pgn'][0])
urls

['Chess.com',
 'https://www.chess.com/openings/Philidor-Defense-3.Bc4',
 'https://www.chess.com/game/live/1051463451']

In [24]:
st = time.time()
def find_opening(mychess_games_pgn_df):
    urls= extractor.find_urls(mychess_games_pgn_df)
    opening_url = list(filter(lambda a: 'openings' in a, urls))
    if opening_url!=[]:
        index = opening_url[0].rfind('/')
        oppening = opening_url[0][index+1:]
    else:
        oppening = ''
    return oppening
mychess_games_df['oppenings'] = mychess_games_df['pgn'].apply(find_opening)
elapsed_time = time.time() - st

In [25]:
mychess_games_df.columns

Index(['index', 'url', 'pgn', 'time_control', 'end_time', 'rated', 'tcn',
       'uuid', 'initial_setup', 'fen', 'time_class', 'rules', 'white.rating',
       'white.result', 'white.@id', 'white.username', 'white.uuid',
       'black.rating', 'black.result', 'black.@id', 'black.username',
       'black.uuid', 'tournament', 'start_time', 'accuracies.white',
       'accuracies.black', 'my_colour', 'my_result', 'my_rating',
       'opponent_rating', 'opponent_username', 'year', 'result',
       'rating_class', 'white_avg_time_per_move', 'black_avg_time_per_move',
       'captured_black_pieces_count', 'captured_white_pieces_count',
       'black_king_checks', 'white_king_checks', 'oppenings'],
      dtype='object')

In [26]:
mychess_games_df.head(3)

,index,url,pgn,time_control,end_time,rated,tcn,uuid,initial_setup,fen,...,year,result,rating_class,white_avg_time_per_move,black_avg_time_per_move,captured_black_pieces_count,captured_white_pieces_count,black_king_checks,white_king_checks,oppenings
0,0,https://www.chess.com/game/live/1051463451,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",300,2015-02-06 11:17:31,True,mC0KgvZRfA3VbsWOlBKBdB!Teg5QBdQGArGrkr6ZsJTJCJ...,2506eeee-5dd6-11df-8000-000000010001,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,8/P5pk/7p/1r6/1P2R3/6P1/1P3P1P/6K1 b - -,...,2015,win,medium,3.108823,4.138235,12,8,4,0,Philidor-Defense-3.Bc4
1,1,https://www.chess.com/game/live/1051466465,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",300,2015-02-06 11:22:16,True,mCYInD0KgvKDfAZReg2Mlt1TvMTMdN8ZN1701N3Vbs!TNx...,95ae0b0a-5dd6-11df-8000-000000010001,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,4r3/1pkq4/p2p3p/2pP2p1/2P1np2/4bR1P/PPP3P1/2Q3...,...,2015,win,medium,4.600000,4.066666,4,6,4,2,Sicilian-Defense-McDonnell-Attack
2,2,https://www.chess.com/game/live/1051469304,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",300,2015-02-06 11:29:20,True,kA0SlB!TbsZJgv5ZmuYIiq90BIZIjzIZcj8!fmJAmAXPeg...,ff2e19d0-5dd6-11df-8000-000000010001,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,6k1/5p1p/pp2p1p1/8/1P1Nn3/r3PP1P/6P1/5RK1 b - -,...,2015,win,medium,5.231250,6.121875,7,8,0,1,Queens-Pawn-Opening-Horwitz-Defense-2.c4


In [27]:
# mychess_games_df.to_csv('preprocessed_data.csv')

In [28]:
mychess_games_df['white.rating']

0       1367
1       1362
2       1537
3       1524
4       1602
        ... 
6405    1627
6406    1569
6407    1543
6408    1563
6409    1569
Name: white.rating, Length: 6410, dtype: int64

In [29]:
mychess_games_df['white.result'].unique()

array(['win', 'resigned', 'timeout', 'insufficient', 'checkmated',
       'repetition', 'abandoned', 'timevsinsufficient', 'agreed',
       'stalemate'], dtype=object)

In [30]:
mychess_games_df.columns

Index(['index', 'url', 'pgn', 'time_control', 'end_time', 'rated', 'tcn',
       'uuid', 'initial_setup', 'fen', 'time_class', 'rules', 'white.rating',
       'white.result', 'white.@id', 'white.username', 'white.uuid',
       'black.rating', 'black.result', 'black.@id', 'black.username',
       'black.uuid', 'tournament', 'start_time', 'accuracies.white',
       'accuracies.black', 'my_colour', 'my_result', 'my_rating',
       'opponent_rating', 'opponent_username', 'year', 'result',
       'rating_class', 'white_avg_time_per_move', 'black_avg_time_per_move',
       'captured_black_pieces_count', 'captured_white_pieces_count',
       'black_king_checks', 'white_king_checks', 'oppenings'],
      dtype='object')

In [31]:
my_white_chess_games = mychess_games_df[mychess_games_df['white.username']==user]
my_black_chess_games = mychess_games_df[mychess_games_df['black.username']==user]

In [32]:
r_fig = px.scatter(my_white_chess_games, x = "white_avg_time_per_move", y="black.rating",color='rating_class')
# r_fig.add_box(x=my_white_chess_games['white_avg_time_per_move'],y=my_white_chess_games['black.rating'])
r_fig.update_layout(height=480,title='my black opponents rating box plot')
r_fig.show()

In [33]:
px.box(my_white_chess_games[my_white_chess_games['white_avg_time_per_move']>0], x = "white_avg_time_per_move", y="rating_class")

In [34]:
import plotly.graph_objects as go


fig = go.Figure()
data = mychess_games_df[mychess_games_df['white_avg_time_per_move']>0]

fig.add_trace(go.Box(y=data[data['time_class']=='rapid']['rating_class'],
                     x=data[data['time_class']=='rapid']['white_avg_time_per_move'],
                     name='rapid',marker_color='green'))

fig.add_trace(go.Box(
y=data[data['time_class']=='blitz']['rating_class'],
x=data[data['time_class']=='blitz']['white_avg_time_per_move'],
name='blitz',
marker_color='yellow'
))

fig.add_trace(go.Box(
y=data[data['time_class']=='bullet']['rating_class'],
x=data[data['time_class']=='bullet']['white_avg_time_per_move'],
name='bullet',
marker_color='blue'
))

fig.update_layout(boxmode='group',title='Rating class vs average time/move',xaxis={'title':'average time/move'},yaxis={'title':'rating class'})
fig.update_traces(orientation='h')
fig.show()


In [35]:
mychess_games_df.columns

Index(['index', 'url', 'pgn', 'time_control', 'end_time', 'rated', 'tcn',
       'uuid', 'initial_setup', 'fen', 'time_class', 'rules', 'white.rating',
       'white.result', 'white.@id', 'white.username', 'white.uuid',
       'black.rating', 'black.result', 'black.@id', 'black.username',
       'black.uuid', 'tournament', 'start_time', 'accuracies.white',
       'accuracies.black', 'my_colour', 'my_result', 'my_rating',
       'opponent_rating', 'opponent_username', 'year', 'result',
       'rating_class', 'white_avg_time_per_move', 'black_avg_time_per_move',
       'captured_black_pieces_count', 'captured_white_pieces_count',
       'black_king_checks', 'white_king_checks', 'oppenings'],
      dtype='object')

In [36]:
r_fig = px.box(mychess_games_df, x = "captured_black_pieces_count", y="opponent_rating",color='rating_class')
r_fig.update_layout(height=500,title='my opponents rating box plot of captured pieces',xaxis={'title':'captured_black_pieces_count'})
r_fig.show()

In [37]:
r_fig = px.scatter(mychess_games_df, y = "captured_black_pieces_count", x="rating_class",color='year')
r_fig.update_layout(height=500,title='my opponents rating box plot of captured pieces',xaxis={'title':'captured_black_pieces_count'})
r_fig.show()

In [38]:
r_fig = px.histogram(mychess_games_df[mychess_games_df['my_colour']=='white'], x = "captured_black_pieces_count")
r_fig.update_layout(height=400,width=600,title='Distribution of me capturing black pieces playing as white',xaxis={'title':'black pieces captured in a single game'},yaxis={'title':'game count'})
r_fig.show()

In [39]:
r_fig = px.histogram(mychess_games_df[mychess_games_df['my_colour']=='black'], x = "captured_white_pieces_count")
r_fig.update_layout(height=400,width=600,title='Distribution of me capturing white pieces playing as black',xaxis={'title':'white pieces captured in a single game'},yaxis={'title':'game count'})
r_fig.show()

In [40]:
print('number of chess games {0} played white: {1}'.format(user,len(my_white_chess_games)))
print('number of chess games {0} played black: {1}'.format(user,len(my_black_chess_games)))

number of chess games venkykasprov played white: 3218
number of chess games venkykasprov played black: 3192


In [41]:
my_black_opponents_rating_df=my_white_chess_games\
                            .groupby(['white.username','time_class'])\
                            .agg(number_games=('black.rating',np.size)\
                                 ,min_black_rating=('black.rating',np.min)\
                                 ,mean_black_rating=('black.rating',np.mean)\
                                 ,median_black_rating=('black.rating',np.median)\
                                 ,max_black_rating=('black.rating',np.max))\
                            .reset_index()
my_black_opponents_rating_df

,white.username,time_class,number_games,min_black_rating,mean_black_rating,median_black_rating,max_black_rating
0,venkykasprov,blitz,1639,839,1703.781574,1711.0,2119
1,venkykasprov,bullet,1303,364,1386.267843,1396.0,1747
2,venkykasprov,rapid,276,1135,1827.351449,1825.5,2052


In [42]:
my_white_opponents_rating_df=my_black_chess_games\
                            .groupby(['black.username','time_class'])\
                            .agg(number_games=('white.rating',np.size)\
                                 ,min_white_rating=('white.rating',np.min)\
                                 ,mean_white_rating=('white.rating',np.mean)\
                                 ,median_white_rating=('white.rating',np.median)\
                                 ,max_white_rating=('white.rating',np.max))\
                            .reset_index()
my_white_opponents_rating_df

,black.username,time_class,number_games,min_white_rating,mean_white_rating,median_white_rating,max_white_rating
0,venkykasprov,blitz,1629,839,1706.324739,1714.0,2268
1,venkykasprov,bullet,1296,628,1388.931327,1397.5,1866
2,venkykasprov,rapid,267,1149,1840.198502,1851.0,2047


In [43]:

r_fig = px.box(my_white_chess_games, x = "time_class", y="black.rating")
r_fig.update_layout(height=480,title='my black opponents rating box plot')
r_fig.show()

In [44]:
ra_fig = px.box(my_black_chess_games, x = "time_class", y="white.rating")
ra_fig.update_layout(height=480,title='my white opponents rating box plot')
ra_fig.show()

In [45]:
my_white_chess_games['result']='lose'
my_white_chess_games['result'] = np.where(my_white_chess_games['white.result']=='win','win',my_white_chess_games['result'])
my_white_chess_games['result'] = np.where((my_white_chess_games['white.result']=='repetition')|(my_white_chess_games['white.result']=='insufficient')|(my_white_chess_games['white.result']=='timevsinsufficient')|(my_white_chess_games['white.result']=='agreed')|(my_white_chess_games['white.result']=='stalemate'),'draw',my_white_chess_games['result'])

In [46]:
my_black_chess_games['result']='lose'
my_black_chess_games['result'] = np.where(my_black_chess_games['black.result']=='win','win',my_black_chess_games['result'])
my_black_chess_games['result'] = np.where((my_black_chess_games['black.result']=='repetition')|(my_black_chess_games['black.result']=='insufficient')|(my_black_chess_games['black.result']=='timevsinsufficient')|(my_black_chess_games['black.result']=='agreed')|(my_black_chess_games['black.result']=='stalemate'),'draw',my_black_chess_games['result'])

In [47]:
my_white_chess_games['white.result'].unique()

array(['win', 'resigned', 'timeout', 'checkmated', 'abandoned',
       'repetition', 'insufficient', 'timevsinsufficient', 'agreed',
       'stalemate'], dtype=object)

In [48]:
my_white_chess_games_results=  my_white_chess_games.groupby(['time_class','white.result','result']).agg(total=('result',np.size)).reset_index()
my_white_chess_games_results.head()

,time_class,white.result,result,total
0,blitz,abandoned,lose,26
1,blitz,agreed,draw,30
2,blitz,checkmated,lose,44
3,blitz,insufficient,draw,12
4,blitz,repetition,draw,45


In [49]:
fig = make_subplots(rows=1, cols=3,  specs=[[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}]])
blitz_result = my_white_chess_games_results[my_white_chess_games_results['time_class']=='blitz']['result'].to_list()
blitz_result_des = my_white_chess_games_results[my_white_chess_games_results['time_class']=='blitz']['white.result'].to_list()
blitz_total = my_white_chess_games_results[my_white_chess_games_results['time_class']=='blitz']['total'].to_list()
bullet_result = my_white_chess_games_results[my_white_chess_games_results['time_class']=='bullet']['result'].to_list()
bullet_total = my_white_chess_games_results[my_white_chess_games_results['time_class']=='bullet']['total'].to_list()
daily_result = my_white_chess_games_results[my_white_chess_games_results['time_class']=='daily']['result'].to_list()
daily_total = my_white_chess_games_results[my_white_chess_games_results['time_class']=='daily']['total'].to_list()
rapid_result = my_white_chess_games_results[my_white_chess_games_results['time_class']=='rapid']['result'].to_list()
rapid_total = my_white_chess_games_results[my_white_chess_games_results['time_class']=='rapid']['total'].to_list()
fig.add_trace(go.Pie(labels=blitz_result,values=blitz_total, title="Blitz"),1,1)
fig.add_trace(go.Pie(labels=bullet_result,values=bullet_total, title="Bullet"),1,2)
# fig.add_trace(go.Pie(labels=daily_result,values=daily_total, title="Daily"),2,1)
fig.add_trace(go.Pie(labels=rapid_result,values=rapid_total, title="Rapid"),1,3)
fig.update_layout(title_text="Win-lose percentage of White")
fig.show()

In [50]:
mychess_games_df_temp=  mychess_games_df.groupby(['time_class','result']).agg(total=('result',np.size)).reset_index()
mychess_games_df_temp.head()

,time_class,result,total
0,blitz,draw,206
1,blitz,lose,1490
2,blitz,win,1572
3,bullet,draw,75
4,bullet,lose,1231


In [51]:
fig = make_subplots(rows=1, cols=3,  specs=[[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}]])
blitz_result = mychess_games_df_temp[mychess_games_df_temp['time_class']=='blitz']['result'].to_list()
blitz_total = mychess_games_df_temp[mychess_games_df_temp['time_class']=='blitz']['total'].to_list()
bullet_result = mychess_games_df_temp[mychess_games_df_temp['time_class']=='bullet']['result'].to_list()
bullet_total = mychess_games_df_temp[mychess_games_df_temp['time_class']=='bullet']['total'].to_list()
rapid_result = mychess_games_df_temp[mychess_games_df_temp['time_class']=='rapid']['result'].to_list()
rapid_total = mychess_games_df_temp[mychess_games_df_temp['time_class']=='rapid']['total'].to_list()
fig.add_trace(go.Pie(labels=blitz_result,values=blitz_total, title="Blitz"),1,1)
fig.add_trace(go.Pie(labels=bullet_result,values=bullet_total, title="Bullet"),1,2)
fig.add_trace(go.Pie(labels=rapid_result,values=rapid_total, title="Rapid"),1,3)
fig.update_layout(title_text="Win-lose ratio for different time class")
fig.show()

In [52]:

oppening_df = mychess_games_df.groupby('oppenings').agg(game_count = ('black.rating',np.size)).reset_index()
oppening_df.sort_values(by = 'game_count',ascending=False).reset_index(drop=True).head(10)

,oppenings,game_count
0,Sicilian-Defense-Taimanov-Variation,204
1,Caro-Kann-Defense-2.Nf3-d5-3.exd5-cxd5-4.d4,155
2,Three-Knights-Opening-3...Bc5,150
3,French-Defense-Knight-Variation-2...d5-3.exd5-...,146
4,Sicilian-Defense-French-Variation,142
5,Philidor-Defense-3.Nc3,106
6,French-Defense-Knight-Variation,100
7,Modern-Defense-with-1-e4-2.Nf3,87
8,Sicilian-Defense-Taimanov-Variation-5.Nc3,78
9,Scandinavian-Defense-Mieses-Kotrc-Variation-3....,70


In [53]:
oppening_df = my_white_chess_games.groupby(['oppenings','result']).agg(game_count = ('white.rating',np.size)).reset_index()

for a in my_white_chess_games.result.unique():
    print(oppening_df[oppening_df.result==a].sort_values(by = 'game_count',ascending=False).head(3))

                                             oppenings result  game_count
917                      Three-Knights-Opening-3...Bc5    win          78
48         Caro-Kann-Defense-2.Nf3-d5-3.exd5-cxd5-4.d4    win          68
185  French-Defense-Knight-Variation-2...d5-3.exd5-...    win          61
                                             oppenings result  game_count
47         Caro-Kann-Defense-2.Nf3-d5-3.exd5-cxd5-4.d4   lose          78
184  French-Defense-Knight-Variation-2...d5-3.exd5-...   lose          70
916                      Three-Knights-Opening-3...Bc5   lose          65
                                             oppenings result  game_count
183  French-Defense-Knight-Variation-2...d5-3.exd5-...   draw          11
806  Sicilian-Defense-Open-Lowenthal-Variation-5.Nx...   draw           9
46         Caro-Kann-Defense-2.Nf3-d5-3.exd5-cxd5-4.d4   draw           9


In [54]:
oppening_df = my_black_chess_games.groupby(['oppenings','result']).agg(game_count = ('black.rating',np.size)).reset_index()

for a in my_black_chess_games.result.unique():
    print(oppening_df[oppening_df.result==a].sort_values(by = 'game_count',ascending=False).head(3))

                                      oppenings result  game_count
1124        Sicilian-Defense-Taimanov-Variation    win          92
1135  Sicilian-Defense-Taimanov-Variation-5.Nc3    win          41
922           Sicilian-Defense-French-Variation    win          39
                                      oppenings result  game_count
1123        Sicilian-Defense-Taimanov-Variation   lose          69
921           Sicilian-Defense-French-Variation   lose          59
1134  Sicilian-Defense-Taimanov-Variation-5.Nc3   lose          31
                                oppenings result  game_count
1122  Sicilian-Defense-Taimanov-Variation   draw           9
920     Sicilian-Defense-French-Variation   draw           7
0                                           draw           3


In [55]:
response = urlopen('https://api.chess.com/pub/player/venkykasprov')
archives_json = json.loads(response.read())
print(archives_json) 

{'avatar': 'https://images.chesscomfiles.com/uploads/v1/user/20878038.c0ef7b26.200x200o.b013b8fc9006.jpeg', 'player_id': 20878038, '@id': 'https://api.chess.com/pub/player/venkykasprov', 'url': 'https://www.chess.com/member/venkykasprov', 'name': 'venkatasubramani karthikeyan', 'username': 'venkykasprov', 'followers': 31, 'country': 'https://api.chess.com/pub/country/IN', 'location': 'Chennai', 'last_online': 1687120554, 'joined': 1422722460, 'status': 'basic', 'is_streamer': False, 'verified': False, 'league': 'Champion'}


In [56]:
mychess_games_df.columns

Index(['index', 'url', 'pgn', 'time_control', 'end_time', 'rated', 'tcn',
       'uuid', 'initial_setup', 'fen', 'time_class', 'rules', 'white.rating',
       'white.result', 'white.@id', 'white.username', 'white.uuid',
       'black.rating', 'black.result', 'black.@id', 'black.username',
       'black.uuid', 'tournament', 'start_time', 'accuracies.white',
       'accuracies.black', 'my_colour', 'my_result', 'my_rating',
       'opponent_rating', 'opponent_username', 'year', 'result',
       'rating_class', 'white_avg_time_per_move', 'black_avg_time_per_move',
       'captured_black_pieces_count', 'captured_white_pieces_count',
       'black_king_checks', 'white_king_checks', 'oppenings'],
      dtype='object')

In [57]:
st = time.time()
import concurrent.futures
import requests
import json

# A dictionary to store the results of previous requests
cache = {}

def find_country(user):
    # Check if the result is already in the cache
    if user in cache:
        return cache[user]
    
    url = f"https://api.chess.com/pub/player/{user}"
    response = requests.get(url)
    archives_json = response.json()
    country_url = archives_json['country']
    
    if country_url in cache:
        return cache[country_url]
    
    response = requests.get(country_url)
    archives_json = response.json()
    country_name = archives_json['name']
    
    # Store the result in the cache
    cache[user] = country_name
    cache[country_url]=country_name
    
    return country_name
mychess_games_df['country'] = mychess_games_df['opponent_username'].apply(find_country)
elapsed_time = time.time() - st
print(elapsed_time)
# # Group the usernames into batches
# batch_size = 10
# batches = [mychess_games_df['opponent_username'][i:i+batch_size] for i in range(0, len(mychess_games_df), batch_size)]

# # Use a ThreadPoolExecutor to make requests in parallel
# with concurrent.futures.ThreadPoolExecutor() as executor:
#     # Iterate over the batches
#     for batch in batches:
#         # Make a list of futures for each username in the batch
#         futures = [executor.submit(find_country, user) for user in batch]
#         # Wait for all the futures to complete
#         results = [future.result() for future in futures]
#         print(len(results))
#         print(len(mychess_games_df.loc[mychess_games_df['opponent_username'].isin(batch)]))
#         # Update the dataframe with the results
#         mychess_games_df.loc[mychess_games_df['opponent_username'].isin(batch), 'country'] = results


986.5763309001923


In [58]:
mychess_games_df['country']

0               India
1         Philippines
2            Slovakia
3             Albania
4             Albania
            ...      
6405            Italy
6406    United States
6407           Brazil
6408            Italy
6409          England
Name: country, Length: 6410, dtype: object

In [59]:
country_count = mychess_games_df.groupby(['country']).agg({'pgn':'count'}).reset_index()
country_count = country_count.rename(columns={'pgn':'player counts'})

In [60]:
country_count

,country,player counts
0,Afghanistan,4
1,Aland Islands,2
2,Albania,50
3,Algeria,10
4,American Samoa,4
...,...,...
176,Wales,5
177,Western Sahara,4
178,Yemen,1
179,Zambia,5


In [61]:
fig = px.choropleth(country_count, 
                    locations='country', 
                    locationmode='country names', 
                    color='player counts', 
                    color_continuous_scale='Blues',
                    range_color=[0, 600])

fig.show()

In [62]:
mychess_games_df = mychess_games_df.sort_values(by='end_time', ascending=False)
mychess_games_df.to_csv('preprocessed_data.csv')